In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from darts import TimeSeries

from darts.metrics import mape, mse, rmse, mae
from darts.models.filtering.moving_average_filter import MovingAverageFilter

from utils import print_metrics, plot_forecast, backtest, Dataset, TimeseriesExperiment

import pickle

import os

In [2]:
FILE_NAME = 'data/electricity.txt'

def get_series_from_dataframe(dataframe, column_name):
  # consts 
  TRAIN_DAYS = 60

  s = TimeSeries.from_dataframe(dataframe, time_col='ds', value_cols=column_name, freq=None, fill_missing_dates=True)
  s = s.slice_n_points_before(s.end_time(), TRAIN_DAYS*24*60)
  s = s.resample('1h', method='pad')

  ma_filter = MovingAverageFilter(12)
  s = ma_filter.filter(s)

  # save to txt file for layer use
  s.to_csv()

  return s

def get_series_from_dataframe_xgboost(df: pd.DataFrame) -> TimeSeries:
  df['Global_active_power'] = pd.to_numeric(df['Global_active_power'], errors='coerce').astype(np.float32)
  df['Global_reactive_power'] = pd.to_numeric(df['Global_reactive_power'], errors='coerce').astype(np.float32)
  df['Voltage'] = pd.to_numeric(df['Voltage'], errors='coerce').astype(np.float32)
  df['Global_intensity'] = pd.to_numeric(df['Global_intensity'], errors='coerce').astype(np.float32)
  df['Sub_metering_1'] = pd.to_numeric(df['Sub_metering_1'], errors='coerce').astype(np.float32)
  df['Sub_metering_2'] = pd.to_numeric(df['Sub_metering_2'], errors='coerce').astype(np.float32)
  df['Sub_metering_3'] = pd.to_numeric(df['Sub_metering_3'], errors='coerce').astype(np.float32)

  # fill NA values with the previous value
  df = df.ffill()

  return get_series_from_dataframe(df, 'Global_active_power')

def load_electricity_data():
  # try to read TimeSeries object from electricity.txt file
  if os.path.exists(FILE_NAME):
    s = TimeSeries.from_csv('data/electricity.txt', time_col='ds', value_cols='rolling_mean_12_Global_active_power')
    return s

  df = pd.read_csv('data/household_power_consumption.txt', sep=';', parse_dates={'ds': ['Date', 'Time']})

  return get_series_from_dataframe_xgboost(df)


## Load dataset

In [3]:
XGBoost_dataset = Dataset(load_electricity_data(), 'electricity')

## Define the model

In [4]:
from darts.models.forecasting.xgboost import XGBModel

xgb_model = XGBModel(
  lags=24 * 7,
  output_chunk_length=100
)


In [5]:
experiments = []
results = []

for horizon in [1, 2, 3, 5, 10]:
  if os.path.exists(f'results/{XGBoost_dataset.name}_{xgb_model.__class__.__name__}_{horizon}.json'):
    print(f'{XGBoost_dataset.name}_{xgb_model.__class__.__name__}_{horizon}.json', 'found - skipping...')
    continue

  xgb_experiment = TimeseriesExperiment(xgb_model, XGBoost_dataset, None, use_pretrained_model=True, forecast_horizon=horizon)
  
  xgb_experiment.run()

/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/utils/parallel.py:114: UserWarning: `sk